In [1]:
from openai import OpenAI
# 设置 API 密钥和组织
api_key = "sk-WO49kazJ9a1PDGbE2lPRhJ9vDG8hak5_mbD60YhIB8T3BlbkFJHbhSLNYAPwzhLHjS72qabHWqGwtBmAvgRpRwN_iYMA"
client = OpenAI(api_key=api_key)

# 设置组织信息
client.organization = "org-34qJHjOz4UGZYVqv2zlohVMe"

In [2]:
import os
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 分析故事内容的函数
def analyze_story_content(text_segment):
    prompt = (
        f"你是一位精通中国民间故事研究的研究员。"
        f"请分析以下故事内容，并准确回答以下问题。"
        f"回答时请直接根据故事内容进行判断，不要发散联想或做出推测。"
        f"回答格式为 1 或 0，其中 1 表示提及，0 表示未提及。\n\n"
        f"1. 故事内容中是否提及畜牧业，例如放牧牛羊？\n"
        f"2. 故事内容中是否提及棉花，例如棉花的种植或使用？\n"
        f"3. 故事内容中是否提及小麦，例如小麦的种植或使用？\n"
        f"4. 故事内容中是否提及渔业或水产养殖，例如捕鱼、养鱼或其他水产活动？\n\n"
        f"请按照以下 JSON 格式返回结果：\n"
        f'{{"畜牧业": 0 or 1, "棉花": 0 or 1, "小麦": 0 or 1, "渔业": 0 or 1}}\n\n'
        f"以下是故事内容：\n{text_segment}"
    )
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=100,
            temperature=0.1,
            top_p=0.9
        )
        result = response.choices[0].message.content.strip()
        
        # 尝试将结果解析为 Python 字典
        parsed_result = json.loads(result)
        return parsed_result
    except Exception as e:
        print(f"Error processing text segment: {e}")
        return {"畜牧业": 0, "棉花": 0, "小麦": 0, "渔业": 0}

# 处理单个文件并返回分析结果
def process_single_file(file_path):
    file_name = os.path.basename(file_path)

    # 读取文件内容
    with open(file_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    # 分析故事内容
    analysis_result = analyze_story_content(text_content)

    # 组织数据行
    row = {
        "文件名称": file_name,
        "文本内容": text_content,
        "畜牧业": analysis_result.get("畜牧业", 0),
        "棉花": analysis_result.get("棉花", 0),
        "小麦": analysis_result.get("小麦", 0),
        "渔业": analysis_result.get("渔业", 0),
    }
    return row

# 处理文件夹中的所有 .txt 文件并分批保存
def process_all_txt_files(input_folder, output_excel_folder):
    # 创建结果存储文件夹
    if not os.path.exists(output_excel_folder):
        os.makedirs(output_excel_folder)
    
    # 获取所有的txt文件
    all_txt_files = []
    for root, _, files in os.walk(input_folder):
        for file_name in files:
            if file_name.endswith('.txt'):
                all_txt_files.append(os.path.join(root, file_name))

    # 使用ThreadPoolExecutor并行处理文件，增加线程数来提高并行度
    data = []
    file_count = 0
    output_batch_count = 1  # 用于命名输出文件

    with tqdm(total=len(all_txt_files), desc="Processing Files") as pbar:
        # 增加最大线程数（如：20）
        with ThreadPoolExecutor(max_workers=40) as executor:
            future_to_file = {executor.submit(process_single_file, file_path): file_path for file_path in all_txt_files}
            
            for future in as_completed(future_to_file):
                result = future.result()
                data.append(result)
                file_count += 1
                
                # 每1000条数据保存一个批次的xlsx文件
                if file_count >= 1000:
                    output_file = os.path.join(output_excel_folder, f"batch_{output_batch_count}.csv")  # 使用CSV格式加快保存速度
                    df = pd.DataFrame(data)
                    df.to_csv(output_file, index=False, encoding='utf-8')  # 使用CSV格式
                    print(f"批次 {output_batch_count} 保存到 {output_file}")
                    
                    # 重置数据列表并更新批次编号
                    data = []
                    file_count = 0
                    output_batch_count += 1
                
                # 更新进度条
                pbar.update(1)

    # 最后保存剩余的数据
    if data:
        output_file = os.path.join(output_excel_folder, f"batch_{output_batch_count}.csv")  # 使用CSV格式
        df = pd.DataFrame(data)
        df.to_csv(output_file, index=False, encoding='utf-8')  # 使用CSV格式
        print(f"批次 {output_batch_count} 保存到 {output_file}")

    # 合并所有批次文件为一个文件
    all_batches = []
    for root, _, files in os.walk(output_excel_folder):
        for file_name in files:
            if file_name.endswith('.csv'):  # 合并CSV文件
                file_path = os.path.join(root, file_name)
                all_batches.append(pd.read_csv(file_path))

    # 合并所有数据
    combined_df = pd.concat(all_batches, ignore_index=True)

    # 最终合并后的文件命名为 '241123validation_question_analysis_results.csv'
    final_output_file = os.path.join(output_excel_folder, '241123validation_question_analysis_results.csv')
    combined_df.to_csv(final_output_file, index=False, encoding='utf-8')  # 使用CSV格式
    print(f"所有结果已合并并保存到 {final_output_file}")

# 指定输入文件夹和输出文件夹路径
input_folder = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales"
output_excel_folder = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results"

# 运行主函数
process_all_txt_files(input_folder, output_excel_folder)

Processing Files:   6%|▌         | 1003/17324 [00:31<10:04, 27.00it/s]

批次 1 保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/batch_1.csv


Processing Files:   7%|▋         | 1193/17324 [00:38<09:48, 27.42it/s]

Error processing text segment: Expecting value: line 1 column 1 (char 0)


Processing Files:  12%|█▏        | 2008/17324 [01:05<08:20, 30.59it/s]

...

批次 16 保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/batch_16.csv


Processing Files:  98%|█████████▊| 17005/17324 [09:44<00:09, 33.32it/s]

批次 17 保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/batch_17.csv


Processing Files: 100%|██████████| 17324/17324 [10:00<00:00, 28.84it/s]


批次 18 保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/batch_18.csv
所有结果已合并并保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/241123validation_question_analysis_results.csv


In [8]:
file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123validation_question_analysis_results/241123validation_question_analysis_results.csv"
df = pd.read_csv(file_path)
df['文件名称'] = df['文件名称'].str.replace('.txt', '', regex=False)
df.shape[0]

17324

剔除 吉林旧 和 少数民族
4）	暂时不要包括少数民族--他们的故事通常是比较不同的，我们之后需要单独研究他们
"E:\坚果云\润平\from MX\少数民族\Merged_Ethnicity_Data_with_English_Names_Xue.csv"
相关少数民族的故事，和对应的ethnic group（in english)。目前阶段只需要把这些故事剔除


In [ ]:

import os
import pandas as pd

# 读取文件夹内所有txt文件的名称（不含.txt）
def get_txt_filenames(folder_path):
    txt_files = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            # 去掉文件扩展名（.txt）
            txt_files.append(os.path.splitext(file_name)[0])
    return txt_files

# 读取CSV文件中的"Name"列
def get_names_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    # 假设CSV中有"Name"列
    return df['name'].tolist()

# 设置路径
folder_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales/吉林596'
csv_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/Merged_Ethnicity_Data_with_English_Names_Xue.csv'

# 获取txt文件名称
txt_filenames = get_txt_filenames(folder_path)

# 获取CSV文件中的名称
csv_names = get_names_from_csv(csv_path)

# 合并两个列表
combined_list = txt_filenames + csv_names
combined_list

['02084__062严羽暑天着羊裘',
 '02081__059李纲巧阻皇帝过沙县',
 '02127__108王绍兰判斗案',
 '02246__228资国寺',
 '02399__386-老鼠借牙',
 '02336__321明姜的传说',
 '02128__109谢瑁樵三难学台',
 '02284__266抓“苏维埃',
 '02028__003,皇天爷和皇天姆造人',
 '02310__294还瑰草',
 '02571__564包国生逗贪吃',
 '02100__079-万人锅',
 '02518__510义盗宋沼',
 '02560__553-寡妇改嫁',
 '02502__493考女婿',
 '02329__313龙眼果和桂圆干',
 '02535__527县太爷选贤人',
 '02113__095大力士吴仔恩',
 '02121__102叫化子喝老爷',
 '02151__132放下屠刀立地成佛',
 '02578__571赵六滩斗知县',
 '02479__469巧妇智斗“鸭肉香',
 '02506__497-两瓮银',
 '02250__232万木林的来历',
 '02066__043风吹松子满山生',
 '02082__060王璞作活画',
 '02297__281五步蛇',
 '02172__153杉竹和合',
 '02279__261-#被',
 '02132__113智除奸细',
 '02348__333“佛跳墙”的传说',
 '02242__224雪峰寺的传说',
 '02281__263陈毅拜师',
 '02557__549冬夜擒贼',
 '02042__019#罗解梦',
 '02389__376红布盖头的由来',
 '02496__487“铁公鸡吃鸡',
 '02237__219通仙桥',
 '02492__483长年智斗财主',
 '02075__053葱补丹田麦补脾',
 '02527__519百忍堂',
 '02228__210望云楼',
 '02201__183-出钱石',
 '02358__344二月二的由来',
 '02280__262草鞋船',
 '02598__591学懒',
 '02212__194银池岗',
 '02516__508三家福',
 '02190__171美人山望台湾',
 '02463__453

In [11]:
df_filtered = df[~df['文件名称'].isin(combined_list)]
df_filtered.shape[0]

12957

In [12]:
df_filtered.head()

,文件名称,文本内容,畜牧业,棉花,小麦,渔业
2,15626__359祭煤窑神节的由来,359.祭煤窑神节的由来\n\nFa\n\n（汉族?盘县特区）\n\n在很多年以前，正月十七...,1,0,0,0
6,15647__381水牛和老虎,381 .水牛和老虎\n\n（壮族?荔波县）\n\n从前有一天，水牛在山坡上吃草，有一只老虎...,1,0,0,0
12,15283__003开天辟地,003.开天辟地\n\n（侗族?三穗县）\n\n天地原来是一大团滚烫的脓脓水水，后来慢慢地变...,0,0,0,0
19,15746__485碰巧成妙对,485.碰巧成妙对\n\n\n\n\n\n\n\n\n\n\n\n从前，有一个财主姓程，家中...,0,0,1,0
139,15796__537两人争官,537.两人争官\n\n（布依方矣?罗句县）\n\n从前，马伍和陈朋两人，一同上京会试，都中...,0,0,0,0


In [15]:
file_path = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales/地图匹配_0426_v4.csv"
df = pd.read_csv(file_path)
merged_df = pd.merge(df, df_filtered, left_on="name", right_on="文件名称", how="left")
merged_df = merged_df.dropna(subset=["文件名称"])
merged_df.head()

,rowid,name,cailu,provgb,citygb,cntygb,area,geocode,PROVGB_1953,CITYGB_1953,...,CITYGB_2010,CNTYGB_2010,new_name,area_group,文件名称,文本内容,畜牧业,棉花,小麦,渔业
4,5,04849__225老鼠的智慧,missing values,missing valuesd,missing valuesd,missing valuesd,宁夏藏抜同仁县,"{'lng': 116.413384, 'lat': 39.910925}",NaN,NaN,...,NaN,NaN,老鼠的智慧,宁夏,04849__225老鼠的智慧,225.老鼠的智慧\n\n（藏抜同仁县）\n\n有一次.老鼠、青蛙、狼，为争夺一件宝物争吵不...,1.0,0.0,0.0,0.0
5,6,06997__285.大人山和弯角顶,黄益新 1987年采录于信宜县,44,4410,440983,广东信宜县,"{'lng': 110.953582, 'lat': 22.360093}",44.0,4463.0,...,NaN,NaN,大人山和弯角顶,广东,06997__285.大人山和弯角顶,285.大人山和弯角顶\n\n（信宜县）\n\n相传很久很久以前，黎村和金洞是个天湖。湖中鱼...,1.0,0.0,0.0,1.0
6,7,07006__294.丹霞山出米洞,黄海燕 1987年8月采录于花县新华镇乐同乡,44,4402,440224,广东仁化县,"{'lng': 113.748627, 'lat': 25.088226}",44.0,4461.0,...,NaN,NaN,丹霞山出米洞,广东,07006__294.丹霞山出米洞,294.丹霞山出米洞\n\n（仁化县）\n\n丹霞山的锦石岩，有一个大雄宝殿，殿旁有一岩壁叫...,0.0,0.0,0.0,0.0
7,8,07168__460.龙门胡须鸡,罗振球男38岁龙门县文化馆干部大专 1986年5月采录于龙门县,44,4413,441324,广东龙门县,"{'lng': 114.259986, 'lat': 23.723894}",44.0,4460.0,...,NaN,NaN,龙门胡须鸡,广东,07168__460.龙门胡须鸡,460.龙门胡须鸡\n\n（龙门县）\n\n相传.在四百多年前，龙门县的一个小山村，有一个美...,1.0,0.0,0.0,0.0
8,9,07474__764.李起初讨批文,missing values,44,4453,445300,广东云浮县,"{'lng': 116.413384, 'lat': 39.910925}",44.0,4460.0,...,NaN,NaN,李起初讨批文,广东,07474__764.李起初讨批文,764.李起初讨批文\n\n（云浮县）\n\n从前，都骑乡山口村有个文人，他写了一篇文章，到...,0.0,0.0,0.0,0.0


In [16]:
columns = ['畜牧业', '棉花', '小麦', '渔业']

# 按 'area_group' 列分组，计算相似度列的平均值
grouped_df = merged_df.groupby('area_group')[columns].mean()
grouped_df

,畜牧业,棉花,小麦,渔业
area_group,,,,
上海,0.167546,0.040897,0.076517,0.205805
云南,0.350000,0.000000,0.200000,0.150000
北京,0.197898,0.015762,0.113835,0.077058
四川,0.350437,0.019651,0.113537,0.117904
天津,0.195933,0.033272,0.118299,0.253235
宁夏,0.416523,0.018933,0.166954,0.044750
安徽,0.225610,0.039024,0.121951,0.158537
山西,0.263514,0.032095,0.135135,0.067568
广东,0.225352,0.023474,0.098592,0.203052


In [17]:
import pandas as pd

# 假设你的 DataFrame 名为 df
# 定义一个映射字典
province_mapping = {
    "上海": "上海市",
    "云南": "云南省",
    "北京": "北京市",
    "吉林": "吉林省",
    "四川": "四川省",
    "天津": "天津市",
    "宁夏": "宁夏回族自治区",
    "安徽": "安徽省",
    "山西": "山西省",
    "广东": "广东省",
    "广西": "广西壮族自治区",
    "新疆": "新疆维吾尔自治区",
    "江苏": "江苏省",
    "江西": "江西省",
    "河北": "河北省",
    "河南": "河南省",
    "浙江": "浙江省",
    "海南": "海南省",
    "湖北": "湖北省",
    "湖南": "湖南省",
    "甘肃": "甘肃省",
    "福建": "福建省",
    "西藏": "西藏自治区",
    "贵州": "贵州省",
    "辽宁": "辽宁省",
    "陕西": "陕西省"
}

# 使用映射字典更新索引
grouped_df.index = grouped_df.index.map(province_mapping)

grouped_df

,畜牧业,棉花,小麦,渔业
area_group,,,,
上海市,0.167546,0.040897,0.076517,0.205805
云南省,0.350000,0.000000,0.200000,0.150000
北京市,0.197898,0.015762,0.113835,0.077058
四川省,0.350437,0.019651,0.113537,0.117904
天津市,0.195933,0.033272,0.118299,0.253235
宁夏回族自治区,0.416523,0.018933,0.166954,0.044750
安徽省,0.225610,0.039024,0.121951,0.158537
山西省,0.263514,0.032095,0.135135,0.067568
广东省,0.225352,0.023474,0.098592,0.203052


In [40]:
import os
from pyecharts.charts import Map
from pyecharts import options as opts

# 生成地图的函数
def generate_map(dataframe, column_name, output_folder):
    """
    根据给定的列生成一张地图并保存到指定文件夹。
    :param dataframe: 数据表
    :param column_name: 需要绘制的列名
    :param output_folder: 保存 HTML 文件的文件夹路径
    """
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)

    # 数据准备
    data = list(zip(dataframe.index, dataframe[column_name]))  # 将地区和数值打包成列表
    
    # 创建地图
    map_chart = (
        Map()
        .add(
            series_name=column_name,  # 显示的图例名称
            data_pair=data,  # 地区与数据对
            maptype="china",  # 地图类型：中国
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title=f"{column_name} 地图"),
            visualmap_opts=opts.VisualMapOpts(
                max_=dataframe[column_name].max(),
                min_=dataframe[column_name].min(),
            range_color=[
                "#f7fbff", "#e6f4fc", "#d2e9f7", "#c2e0f7", "#a3c9f2", 
                "#85b2ed", "#669ce8", "#4a88e3", "#3f75a5", "#2c5b88", 
                "#234c72", "#1c3b5d"
            ]
           ),
        )
    )
    
    # 保存为 HTML 文件，并设置宽高
    output_path_html = os.path.join(output_folder, f"{column_name}_map.html")
    map_chart.render(output_path_html, width="1200px", height="800px")  # 通过 render 设置宽高
    print(f"{column_name} 的地图已生成并保存到 {output_path_html}！")

# 指定输出文件夹
output_folder_html = "/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123map_validation_questions"

# 遍历 grouped_df 的每一列生成地图
for column in grouped_df.columns:
    generate_map(grouped_df, column, output_folder_html)

畜牧业 的地图已生成并保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123map_validation_questions/畜牧业_map.html！
棉花 的地图已生成并保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123map_validation_questions/棉花_map.html！
小麦 的地图已生成并保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123map_validation_questions/小麦_map.html！
渔业 的地图已生成并保存到 /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241123map_validation_questions/渔业_map.html！


In [25]:
pip install snapshot-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 8.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.
